In [1]:
import numpy as np
import csv
import matplotlib.pyplot as plt
import math
import random
import pandas as pd

import seaborn as sns
from __future__ import division
import time
from cvxpy import *

from distutils.log import error
import errno
import numpy as np
import pandas as pd
import time
from datetime import date
import cvxpy as cp

import parameters

In [2]:
User_PV_Data = pd.read_csv('./Solar_Demand.csv')

# get_all_values gives a list of rows.


userdata = np.array(User_PV_Data.values)


print(User_PV_Data)

     Time  PV_Output  PV_Percentage  Demand_kW  Grid_Cost  Unnamed: 5  \
0    0:00        0.0            0.0       0.12       0.01         NaN   
1    0:15        0.0            0.0       0.12       0.01         NaN   
2    0:30        0.0            0.0       0.12       0.01         NaN   
3    0:45        0.0            0.0       0.12       0.01         NaN   
4    1:00        0.0            0.0       0.16       0.01         NaN   
..    ...        ...            ...        ...        ...         ...   
91  22:45        0.0            0.0       0.20       0.01         NaN   
92  23:00        0.0            0.0       0.16       0.01         NaN   
93  23:15        0.0            0.0       0.16       0.01         NaN   
94  23:30        0.0            0.0       0.16       0.01         NaN   
95  23:45        0.0            0.0       0.16       0.01         NaN   

    Unnamed: 6  
0          NaN  
1          NaN  
2          NaN  
3          NaN  
4          NaN  
..         ...  
91  

In [3]:
Battery_Data = pd.read_csv('./BattData.csv', index_col=0)

# get_all_values gives a list of rows.


#data = np.array(Battery_Data.values)


Battery_Data



,B_bar,B_P,eta_b_c,eta_b_d
ID,,,,
1,200,50.0,0.95,0.95
2,300,75.0,0.95,0.95
3,600,150.0,0.95,0.95
4,100,25.0,0.95,0.95
5,100,25.0,0.95,0.95
6,10,2.5,0.95,0.95
7,1500,375.0,0.95,0.95
8,400,100.0,0.95,0.95
9,10,2.5,0.95,0.95


In [4]:
site_id = 6
BattParam = Battery_Data.loc[site_id]
BattParam

B_bar      10.00
B_P         2.50
eta_b_c     0.95
eta_b_d     0.95
Name: 6, dtype: float64

In [5]:
class Battery(object):
    """ Used to store information about the battery.
       :param current_charge: is the initial state of charge of the battery
       :param capacity: is the battery capacity in kWh
       :param charging_power_limit: the limit of the power that can charge the battery in kW
       :param discharging_power_limit: the limit of the power that can discharge the battery in kW
       :param battery_charging_efficiency: The efficiecny of the battery when charging
       :param battery_discharing_efficiecny: The discharging efficiency
    """
    def __init__(self,
                 current_charge=0.0,
                 capacity=0.0,
                 charging_power_limit=1.0,
                 discharging_power_limit=-1.0,
                 charging_efficiency=0.95,
                 discharging_efficiency=0.95):
        self.current_charge = current_charge
        self.capacity = capacity
        self.charging_power_limit = charging_power_limit
        self.discharging_power_limit = discharging_power_limit
        self.charging_efficiency = charging_efficiency
        self.discharging_efficiency = discharging_efficiency

In [6]:
batt_id = 1
# use battery class to store information
batt = Battery(capacity=BattParam["B_bar"], charging_power_limit=BattParam["B_P"], 
               discharging_power_limit=-1*BattParam["B_P"], 
               charging_efficiency=BattParam["eta_b_c"], 
               discharging_efficiency=BattParam["eta_b_d"])

batt.capacity

10.0

In [7]:
prof=pd.Series(dtype='object')

prof.loc['Solar']=User_PV_Data.iloc[:, 1]
prof.loc['Load']=User_PV_Data.iloc[:, 3]
prof.loc['Price']=User_PV_Data.iloc[:, 4]*100

N=prof.Load.shape[0]

In [8]:
prof.Load

0     0.12
1     0.12
2     0.12
3     0.12
4     0.16
      ... 
91    0.20
92    0.16
93    0.16
94    0.16
95    0.16
Name: Demand_kW, Length: 96, dtype: float64

In [46]:
net = prof.Load-prof.Solar
# split load into +ve and -ve
posLoad = np.copy(net)
negLoad = np.copy(net)
for j,e in enumerate(net):
    if e>=0:
        negLoad[j]=0
    else:
        posLoad[j]=0
posLoadDict = dict(enumerate(posLoad))
negLoadDict = dict(enumerate(negLoad))

In [47]:
import pyomo.environ as pyo

In [48]:
m = pyo.ConcreteModel()

m.Time = pyo.Set(initialize=pyo.RangeSet(N),ordered=True)

#scalar parameters
m.Kpv = pyo.Param(initialize=.03) # Solar capacity cost ($/kW)
m.Kb = pyo.Param(initialize= .25) # Battery capacity cost ($/kWh)
m.VCC = pyo.Param(initialize= 75) # Battery discharge cost ($/MWh)
m.nu =pyo.Param(initialize= 0.85) # Battery efficiency
m.r = pyo.Param(initialize= 0.004) # Real interest rate
m.b_min_cap = pyo.Param(initialize= 0) # min battery cap

#time indexed parameters
m.grid_cost = pyo.Param(m.Time,initialize=dict(enumerate(prof.Price,1)),within=pyo.Any)
m.demand = pyo.Param(m.Time,initialize=dict(enumerate(prof.Load,1)),within=pyo.Any)
m.solar = pyo.Param(m.Time,initialize=dict(enumerate(prof.Solar,1)),within=pyo.Any)

In [49]:
#time indexed Variables
m.SOC= pyo.Var(m.Time, bounds=(0,batt.capacity), initialize=batt.capacity) # Battery state of charge at step k (MWh)
m.Bc = pyo.Var(m.Time, initialize=0) # Battery charge at step k (MWh)
m.Bd = pyo.Var(m.Time, initialize=0, domain=pyo.NonNegativeReals) # Battery dischcharge at step k (MWh)
m.Gi = pyo.Var(m.Time, initialize = dict(enumerate(posLoad,1)), domain=pyo.NonNegativeReals) # Grid energy import at step k (MWh)
m.Ge = pyo.Var(m.Time, initialize = dict(enumerate(negLoad,1)), domain=pyo.NonNegativeReals) 
m.Dpv = pyo.Var(m.Time, domain=pyo.NonNegativeReals) # Curtailed PV production at step k (MWh)
m.Dl =pyo.Var(m.Time, domain=pyo.NonNegativeReals) # Cutailed load at step k (MWh)
m.X =pyo.Var(m.Time, domain=pyo.NonNegativeReals) # Solar capacity MW
m.Y = pyo.Var(m.Time, domain=pyo.NonNegativeReals) # Battery capacity MWh


In [72]:
#CONSTRAINTS
#Purchase constraint
def purchase_constraint(m,k):
    return m.Gi[k] == m.demand[k] - m.Bd[k] + m.Bc[k] - m.X[k]*m.solar[k] + m.Ge[k]

def panel_constraint(m,k):
     return m.X[k]*m.solar[k]> 0

#State of charge constraint
def soc_constraint(m,k):
#     m.SOC[0]==Y
    if k<=95:
        return (m.SOC[k+1]==m.SOC[k] + m.Bc[k]*m.nu - m.Bd[k]*m.nu)
    else:
        return (m.SOC[k]==m.SOC[k] + m.Bc[k]*m.nu - m.Bd[k]*m.nu)
    
#Discharge and charge constraints
def discharge_constraint_1(m,k):
    """ Maximum discharge rate within time set k"""
    return m.Bd[k] <= m.Y[k]/4

def discharge_constraint_2(m,k):
    """ Sets the maximum energy available to be discharged as the SOC - minimum SOC """
    return m.Bd[k] <= m.SOC[k] 

def charge_constraint_1(m,k):
    """ Maximum charge rate within a single hour """
    return m.Bc[k] <= m.Y[k]/4

def charge_constraint_2(m,k):
    """ Sets the maximum energy available to be cahrge as the SOC max """
    return m.Bc[k]  <= m.Y[k] - m.SOC[k]



In [73]:
#constraints
m.Gi_c = pyo.Constraint(m.Time, rule = purchase_constraint)
m.SOC_c = pyo.Constraint(m.Time, rule = soc_constraint)
m.Bd_c1 = pyo.Constraint(m.Time, rule = discharge_constraint_1)
m.Bd_c2 = pyo.Constraint(m.Time,rule = discharge_constraint_2)
m.Bc_c1 = pyo.Constraint(m.Time,rule = charge_constraint_1)
m.Bc_c2 = pyo.Constraint(m.Time,rule = charge_constraint_2)
m.X_c = pyo.Constraint(m.Time,rule = panel_constraint)
#OBJECTIVE 
#m.objective = pyo.Objective(expr = sum(SOC[k].value for k in m.Time), sense = pyo.minimize)   

m.objective = pyo.Objective(expr = sum([(m.Gi[k].value*m.grid_cost[k]) for k in m.Time]), sense = pyo.minimize)   
 
opt = pyo.SolverFactory('glpk')
results = opt.solve(m)
results.write()
 
x=[pyo.value(m.X[1])]
y=[pyo.value(m.Y[1])]

for t in m.Time:
    if t < N:
        x.append(pyo.value(m.X[t+1]))
        y.append(pyo.value(m.Y[t+1]))
plt.plot(x)

    'pyomo.core.base.constraint.IndexedConstraint'>) on block unknown with a
    new Component (type=<class
    'pyomo.core.base.constraint.IndexedConstraint'>). This is usually
    indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().
    'pyomo.core.base.constraint.IndexedConstraint'>) on block unknown with a
    new Component (type=<class
    'pyomo.core.base.constraint.IndexedConstraint'>). This is usually
    indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().
    'pyomo.core.base.constraint.IndexedConstraint'>) on block unknown with a
    new Component (type=<class
    'pyomo.core.base.constraint.IndexedConstraint'>). This is usually
    indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().
    'pyomo.core.base.constraint.IndexedConstraint'>) on block unknown with a
    new Component (type=<class
    'py

ValueError: Invalid constraint expression. The constraint expression resolved to a trivial Boolean (False) instead of a Pyomo object. Please modify your rule to return Constraint.Infeasible instead of False.

Error thrown for Constraint 'X_c[1]'